In [319]:
import pandas as pd
import numpy as np

Reading in the training and test data we well as the column name to column label mapping file created in `column_name.ipynb`

In [386]:
codebook=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\column_name_file_clean.csv")
train=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\train.csv")
test=pd.read_csv(r"C:\Users\Jack\Desktop\Project441\test.csv")

C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,4,6,10,12,18,24,35,56,57,65,85,86,87,88,89,98,99,100,105,124,125,126,127,128,130,131,133,166,171,172,192,193,194,195,204,205,206,207,228,229,230,240,256,257,258,259,260,261,262) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Jack\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,6,10,12,21,24,35,85,87,88,89,100,124,126,127,128,130,131,133,166,172,177,193,194,195,205,206,207,220,222,228,229,230,240,257,258,260,261,262) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


For all columns, values of `.a,.b,.c,.d` or  `.` have different meanings, typically `Nothing, Not applicable, Refusal, Don't know` or `No answer`. 

For we create an indicator variable for each column specifying whether each respondant responded with any of these values. For the original column, we replace this `.a,.b,.c,.d` or  `.` value with `na`.

For Example: Respondant 9948 has responce `.a` for `v263: Year of birth of second person in household`. So we create column `v263given`  9948 will have value 1 `v263` 9948 will now have value `NaN`. Respondents who did not have `.a,.b,.c,.d` or  `.` provided for `v263` will have 0 in `v263given` and the original value in `v263`.

In [338]:
for c in train.columns.drop('satisfied'):
    train[c+'given']=train[c].isin(['.a','.d','.c','.b','.'])*1
    train.loc[train[c].isin(['.a','.d','.c','.b','.']),c]=np.nan
    test[c+'given']=test[c].isin(['.a','.d','.c','.b','.'])*2
    test.loc[test[c].isin(['.a','.d','.c','.b','.']),c]=np.nan

Values are sometimes recorded in two datatypes. for example we observe the unique values for `v10: Born in country`:

In [339]:
train['v10'].unique()

array([1, 2, '1', '2', nan], dtype=object)

We see 1 and '1' are both possible values, to correct for this we decide to change the string values to be numerics:

In [340]:
for c in train.columns.drop('satisfied'):
    try:
        train[c]=train[c].astype(float)
        
        ### Addition
        test[c]=test[c].astype(float)
        ####
    except ValueError:
        print(c+" is not numeric")

v17 is not numeric
v20 is not numeric
v25 is not numeric
v78 is not numeric
v154 is not numeric
v155 is not numeric
v161 is not numeric
cntry is not numeric


We can now check again to make sure this issue has been corrected:

In [341]:
train['v10'].unique()

array([ 1.,  2., nan])

All variables which are not `float` do not need to be imputed as NA can be left as a value which will be converted into a dummy variable.

In [342]:
train.columns[train.dtypes!='float64']

Index(['v17', 'v20', 'v25', 'v78', 'v154', 'v155', 'v161', 'cntry',
       'satisfied'],
      dtype='object')

Make a separate training set with only numeric values

In [365]:
num_col=train.columns[~train.columns.isin(train.columns[train.dtypes!='float64'])]
train_n=train[num_col]
train_n=train_n.drop(columns={'id'})

#### Addition
num_col=test.columns[~test.columns.isin(test.columns[test.dtypes!='float64'])]
test_n=test[num_col]
test_n=test_n.drop(columns={'id'})

In [366]:
train_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v262given,v263given,v264given,v265given,v266given,v267given,v268given,v269given,v270given,cntrygiven
0,2.0,2.0,74.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,4.0,2.0,58.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,6.0,2.0,47.0,11010.0,11010.0,2.0,2.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
3,10.0,2.0,22.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,0.0,1.0,24.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


Using `Imputer` from `sklearn` we impute these numerical columns with the mean.

In [367]:
test_n.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v262given,v263given,v264given,v265given,v266given,v267given,v268given,v269given,v270given,cntrygiven
0,0.0,1.0,63.0,15020.0,NaN,2.0,2.0,1.0,2.0,1.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0
1,5.0,2.0,44.0,21070.0,21070.0,1.0,2.0,2.0,1.0,2.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0
2,10.0,2.0,21.0,11010.0,11010.0,2.0,1.0,2.0,2.0,1.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0
3,2.0,2.0,75.0,11010.0,NaN,2.0,2.0,2.0,2.0,1.0,...,0.0,0.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0
4,0.0,1.0,22.0,11010.0,11010.0,2.0,2.0,2.0,2.0,1.0,...,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0


In [368]:
test_n.nunique()[test_n.nunique()==0]

v89     0
v195    0
v207    0
v262    0
dtype: int64

In [369]:
from sklearn.preprocessing import Imputer
imputer = Imputer()
train_n=imputer.fit_transform(train_n)
train_n=pd.DataFrame(train_n)
train_n.columns=num_col[1:]

## Addition
for v in ['v89','v195','v207','v262']:
    test_n[v]=(train_n[v]).mean()
test_n=imputer.fit_transform(test_n)
test_n=pd.DataFrame(test_n)

test_n.columns=num_col[1:]

Make a separate training set with only the character variables:

In [370]:
train_c=train[train.columns[(train.dtypes!='float64')&(train.columns!='satisfied')]]

## Addition
test_c=train[test.columns[(test.dtypes!='float64')&(test.columns!='satisfied')]]

For these character variables, we convert them to dummy variables where each level will have it's own variable with levels:
- 0 if the respondent did not give that level 
- 1 if the respondent gave that level 

In [371]:
train_c=pd.get_dummies(train_c)

## Addition
test_c=pd.get_dummies(test_c)

We now append all these dummy variable columns to our `train_n` dataframe to create a finalized training set, with no missing data in it. 

In [372]:
for c in train_c.columns:
    train_n[c]=train_c[c]
    
    ## Addition
    test_n[c]=test_c[c]

In [373]:
y_train=train['satisfied']

,id,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v262given,v263given,v264given,v265given,v266given,v267given,v268given,v269given,v270given,cntrygiven
0,9948.0,2.0,2.0,74.0,11010.0,NaN,2.0,2.0,2.0,2.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,25601.0,4.0,2.0,58.0,11010.0,NaN,2.0,2.0,2.0,2.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,8592.0,6.0,2.0,47.0,11010.0,11010.0,2.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
3,29593.0,10.0,2.0,22.0,11010.0,NaN,2.0,2.0,2.0,2.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,4252.0,0.0,1.0,24.0,11010.0,NaN,2.0,2.0,2.0,2.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [315]:
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [316]:
gbm = GradientBoostingClassifier(
    n_estimators= 500,
    max_depth= 3,
    learning_rate= 0.02,
    loss='deviance')

In [318]:
gbm.fit(train_n,y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.02, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [290]:
import seaborn as sns

In [ ]:
(pd.Series(model.feature_importances_, index=X.columns)
   .nlargest(4)
   .plot(kind='barh')) 

In [374]:
imp=pd.DataFrame({'name':train_n.columns,'imp':gbm.feature_importances_})

In [427]:
imp.sort_values('imp',ascending=False).tail(100)

,name,imp
576,v17_CU,0.0
567,v17_CF,0.0
575,v17_CR,0.0
574,v17_CO,0.0
573,v17_CN,0.0
572,v17_CM,0.0
571,v17_CL,0.0
570,v17_CI,0.0
569,v17_CH,0.0
568,v17_CG,0.0


In [411]:
pred=pd.DataFrame(gbm.predict_proba(test_n))

In [418]:
pred['id']=test['id']

In [423]:
pd.DataFrame({'id':pred['id'],'prob':pred[1]}).to_csv(r"C:\Users\Jack\Desktop\Project441\pred.csv")

In [7]:
train['num_in_house']=train.iloc[:,258:271].max(axis=1)

In [8]:
train.columns[258:271]

Index(['v258', 'v259', 'v260', 'v261', 'v262', 'v263', 'v264', 'v265', 'v266',
       'v267', 'v268', 'v269', 'v270'],
      dtype='object')

In [9]:
train.columns[258:271][6]

'v264'

In [10]:
train['num_in_house2']=0
for c in range(0, 13):
    train['num_in_house2']=train['num_in_house2']+(train[train.columns[258:271][c]]==train['num_in_house'])*[1,10,11,12,13,2,3,4,5,6,7,8,9][c]

In [12]:
v='cntry'

g=pd.DataFrame(train.groupby([v])['satisfied'].mean())
g=g.reset_index()
g2=pd.DataFrame(train.groupby([v])['satisfied'].count())
g2=g2.rename(columns={'satisfied':'n'})
g2=g2.reset_index()
g=g.merge(g2)
g.sort_values('satisfied',ascending=False)

,cntry,satisfied,n
5,DK,0.816429,1193
8,FI,0.732002,1653
2,CH,0.729184,1189
16,NO,0.696113,1132
19,SE,0.692469,1434
15,NL,0.627164,1502
4,DE,0.604690,2388
13,IL,0.580437,2014
0,AT,0.570296,1387
1,BE,0.555556,9


In [13]:
g2

,cntry,n
0,AT,1387
1,BE,9
2,CH,1189
3,CZ,1683
4,DE,2388
5,DK,1193
6,EE,1606
7,ES,1520
8,FI,1653
9,FR,1502


In [262]:
import sklearn
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import numpy as np

In [263]:
target = 'satisfied'
IDcol = 'id'

In [264]:
gbm = GradientBoostingClassifier(
    n_estimators= 500,
    max_depth= 3,
    learning_rate= 0.02,
    loss='deviance')


In [184]:
from sklearn.preprocessing import Imputer
imputer = Imputer()
imputer.fit_transform(train)

In [30]:
gbm.feature_importances_

array([0.14177209, 0.13667957, 0.72154834])

In [ ]:
gbm.score(X_val, y_val)

In [37]:
train['v10'].unique()

array(['1', '2', 'nan'], dtype=object)

In [58]:
a=gbm.predict(pd.get_dummies(train['v10','v79']))

In [59]:
pd.Series(a).value_counts()

1    30073
0        7
dtype: int64

In [62]:
train['satisfied'].value_counts()

1    15626
0    14454
Name: satisfied, dtype: int64